In [2]:
import pandas as pd
import numpy as np
import validation
import math
data=pd.read_csv('SPEC.csv')
data=np.array(data)
np.random.shuffle(data)

In [3]:
n_features=len(data[0])-1
n_data=len(data)

In [4]:
#Iris-setosa
for i in range(n_data):
    if(data[i][n_features]=="Yes"):
        data[i][n_features]=1
    else:
        data[i][n_features]=0

In [5]:
thresold=0.5
n_epochs=1000
learning_rate=0.3
k_fold=10
layer=[n_features,5,1]
eps=0.001
weights=[[[1/(layer[i]+layer[i+1])]*layer[i] for j in range(layer[i+1])] for i in range(len(layer)-1)]
bias=[[1]*layer[i+1] for i in range(len(layer)-1)]
nodes=[[0]*layer[i] for i in range(len(layer))]
error=[[0]*layer[i] for i in range(len(layer))]

def function(row,nodes,weights,bias,thresold,layer):
    for i in range(len(row)):
        nodes[0][i]=row[i]
    for lay in range(1,len(layer)):
        for j in range(layer[lay]):
            y=bias[lay-1][j]
            for k in weights[lay-1][j]:
                y=y+nodes[lay-1][j]*k
            nodes[lay][j]=1/(1+math.exp(-y))
#     print(nodes[len(layer)-1])
    if(nodes[len(layer)-1][0]>=thresold):
        y=1
    else:
        y=0
    return nodes,y

def valid(z,cost,a,b,c,d):
    if cost==0:
        if z==1:
            return a+1,b,c,d
        else:
            return a,b,c+1,d
    else:
        if z==0:
            return a,b+1,c,d
        else:
            return a,b,c,d+1

def kfolddata(j,data):
    x_test=data[j*k_fold:(j+1)*k_fold,0:n_features]
    y_test=data[j*k_fold:(j+1)*k_fold,n_features]
    if(j+2<=k_fold):
        x_train=data[(j+1)*k_fold:,0:n_features]
        y_train=data[(j+1)*k_fold:,n_features]
        if(j-1>=0):
            x_train=np.append(x_train,data[0:j*k_fold,0:n_features],axis=0)
            y_train=np.append(y_train,data[0:j*k_fold,n_features],axis=0)
    else:
        x_train=data[0:j*k_fold,0:n_features]
        y_train=data[0:j*k_fold,n_features]
    return x_train,y_train,x_test,y_test
    
def back_prop(weights,nodes,row,thresold,bias,learning_rate,cost,error,z):
    for i in range(len(row)):
        nodes[0][i]=row[i]
    #output layer error
    error[len(error)-1][0]=(z-nodes[len(error)-1][0])*(1-nodes[len(error)-1][0])*nodes[len(error)-1][0]
#     print(error[len(error)-1][0])
    #hidden layers
    for lay in range(len(error)-2,0,-1):
        for nod in range(len(nodes[lay])):
            err=0
            for nxtlaynod in range(len(nodes[lay+1])):
                err+=error[lay+1][nxtlaynod]*weights[lay][nxtlaynod][nod]             
            error[lay][nod]=nodes[lay][nod]*(1-nodes[lay][nod])*err*-1
    #update weights
    for lay in range(len(weights)):
        for j in range(len(weights[lay])):
            for i in range(len(weights[lay][j])):
                weights[lay][j][i]+=learning_rate*nodes[lay][i]*error[lay+1][j]
    for lay in range(len(bias)):
        for i in range(len(bias[lay])):
            bias[lay][i]+=learning_rate*error[lay+1][i]
    return weights,bias

def printfoldresult(test_TT,test_TF,test_FF,test_FT,eps):
    print("trainTT:",train_TT," trainTF:",train_TF," trainFF:",train_FF," trainFT:",train_FT)
    print("train_accuracy",(train_TT+train_FF)/(train_TT+train_TF+train_FF+train_FT+eps))
    print("train Precision +:",(train_TT)/(train_TT+train_FT+eps))
    print("train Precision -:",(train_FF)/(train_FF+train_TF+eps))
    print("train recall +:",(train_TT)/(train_TT+train_TF+eps))
    print("train recall -:",(train_FF)/(train_FF+train_FT+eps))
    print("testTT:",test_TT," testTF:",test_TF," testFF:",test_FF," testFT:",test_FT)
    print("test_accuracy",(test_TT+test_FF)/(test_TT+test_TF+test_FF+test_FT+eps))
    print("test Precision +:",(test_TT)/(test_TT+test_FT+eps))
    print("test Precision -:",(test_FF)/(test_FF+test_TF+eps))
    print("test recall +:",(test_TT)/(test_TT+test_TF+eps))
    print("test recall -:",(test_FF)/(test_FF+test_FT+eps))

def averageresults(train_TT,train_TF,train_FF,train_FT,test_TT,test_TF,test_FF,test_FT,eps,accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg):
    accTra+=(train_TT+train_FF)/(train_TT+train_TF+train_FF+train_FT+eps)
    precTraPos+=((train_TT)/(train_TT+train_FT+eps))
    precTraNeg+=((train_FF)/(train_FF+train_TF+eps))
    recTraPos+=((train_TT)/(train_TT+train_TF+eps))
    recTraNeg+=((train_FF)/(train_FF+train_FT+eps))
    accTes+=((test_TT+test_FF)/(test_TT+test_TF+test_FF+test_FT+eps))
    precTesPos+=((test_TT)/(test_TT+test_FT+eps))
    precTesNeg+=((test_FF)/(test_FF+test_TF+eps))
    recTesPos+=((test_TT)/(test_TT+test_TF+eps))
    recTesNeg+=((test_FF)/(test_FF+test_FT+eps))
    return accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg 

def printepochresult(accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg,k_fold):
    print("average result for ",k_fold,"fold")
    print("train_accuracy:",accTra/k_fold)
    print("train Precision +:",precTraPos/k_fold)
    print("train Precision -:",precTraNeg/k_fold)
    print("train recall +:",recTraPos/k_fold)
    print("train recall -:",recTraNeg/k_fold)
    print("test_accuracy",accTes/k_fold)
    print("test Precision +:",precTesPos/k_fold)
    print("test Precision -:",precTesNeg/k_fold)
    print("test recall +:",recTesPos/k_fold)
    print("test recall -:",recTesNeg/k_fold)
    
for i in range(n_epochs):
    print("epoch no",i+1,"+"*60)
    accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg=(0 for i in range(10))
    for j in range(int(n_data/k_fold)):
#         print("fold no:",j)
        x_train,y_train,x_test,y_test=kfolddata(j,data)
        #validation variables
#         train_TT,train_TF,train_FF,train_FT,test_TT,test_TF,test_FF,test_FT=(0 for i in range(8))
        for i in range(len(x_train)):
            nodes,y=function(x_train[i],nodes,weights,bias,thresold,layer)
            z=y_train[i]
            cost=z-y
#             train_TT,train_TF,train_FF,train_FT=valid(y,cost,train_TT,train_TF,train_FF,train_FT)
            weights,bias=back_prop(weights,nodes,x_train[i],thresold,bias,learning_rate,cost,error,z)
#             print(weights)
        for i in range(len(x_test)):
            nodes,y=function(x_test[i],nodes,weights,bias,thresold,layer)
            z=y_test[i]
            cost=z-y
#             test_TT,test_TF,test_FF,test_FT=valid(y,cost,test_TT,test_TF,test_FF,test_FT)
#         printfoldresult(test_TT,test_TF,test_FF,test_FT,eps)
#         accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg=averageresults(train_TT,train_TF,train_FF,train_FT,test_TT,test_TF,test_FF,test_FT,eps,accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg)
    printepochresult(accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg,int(n_data/k_fold))


epoch no 1 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


IndexError: list index out of range